In [1]:
from htm.bindings.sdr import SDR
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
import copy

In [2]:
class ReflexiveMemory:
  def __init__(self):
    self.previous_sdr = None
    self.pairs = {}

  def add(self, sdr):
    current_sdr = '-'.join(map(str, sdr))
    if(self.previous_sdr != None):
      complex_key = self.previous_sdr + ' ' + current_sdr
      pair_count = self.pairs.get(complex_key, 0)
      self.pairs[complex_key] = pair_count + 1
    self.previous_sdr = current_sdr

  def find(self, sdr):
    current_sdr = '-'.join(map(str, sdr))
    return_key = None
    return_value = 0
    for k, v in self.pairs.items():
      if(k.split(' ')[0] == current_sdr and v > return_value):
        return_key = k.split(' ')[1]
        return_value = v
    return return_value, return_key


In [3]:
inputDimensions = (32, 32)
activeDimensions = (64, 64)

config = {
    'sp': {
        'inputDimensions': inputDimensions,
        'columnDimensions': activeDimensions,
        'potentialPct': 0.85,
        'potentialRadius': inputDimensions[0] * inputDimensions[1],
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    }
}


In [4]:
inputSDR  = SDR(dimensions = inputDimensions)
activeSDR = SDR(dimensions = activeDimensions)

sp = SpatialPooler(
    inputDimensions = config['sp']['inputDimensions'],
    columnDimensions = config['sp']['columnDimensions'],
    potentialPct = config['sp']['potentialPct'],
    potentialRadius = config['sp']['potentialRadius'],
    globalInhibition = config['sp']['globalInhibition'],
    localAreaDensity = config['sp']['localAreaDensity'],
    synPermInactiveDec = config['sp']['synPermInactiveDec'],
    synPermActiveInc = config['sp']['synPermActiveInc'],
    synPermConnected = config['sp']['synPermConnected'],
    boostStrength = config['sp']['boostStrength'],
    wrapAround = config['sp']['wrapAround'],
    seed = config['sp']['seed']
  )

tm = TemporalMemory(
    columnDimensions = config['sp']['columnDimensions'],
    cellsPerColumn = config['tm']['cellsPerColumn'],
    activationThreshold = config['tm']['activationThreshold'],
    initialPermanence = config['tm']['initialPermanence'],
    connectedPermanence = config['sp']['synPermConnected'],
    minThreshold = config['tm']['minThreshold'],
    maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
    permanenceIncrement = config['tm']['permanenceIncrement'],
    permanenceDecrement = config['tm']['permanenceDecrement'],
    predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
    maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
    maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
  )

rm = ReflexiveMemory()

In [5]:
inputSequence = []
for i in range(10):
    inputSDR.randomize(0.02)
    inputSequence.append(copy.deepcopy(inputSDR.sparse))

inputSequence.append(inputSequence[0])
inputSequence.append(inputSequence[1])
inputSequence.append(inputSequence[2])

In [6]:
for i in inputSequence:
    inputSDR.sparse = i
    sp.compute(inputSDR, config['sp']['learn'], activeSDR)
    rm.add(copy.deepcopy(activeSDR.sparse))
    tm.compute(activeSDR, learn=config['tm']['learn'])


In [7]:
rm.pairs.values()

dict_values([2, 2, 1, 1, 1, 1, 1, 1, 1, 1])